In [1]:
import massiveOES
# import piUtils
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage.filters import gaussian_filter1d
import scipy.optimize
from scipy.constants import physical_constants, h, c
from collections import OrderedDict
# import sys
from PIL import Image
# sys.path.append(r'c:\users\lukas\envs\py38\lib\site-packages')

%matplotlib qt5

C:\Users\kusyl\AppData\Local\Temp\ipykernel_16616\3178636085.py:5: DeprecationWarning: Please use `gaussian_filter1d` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter1d


In [2]:
def read_tif(path):
    """
    path - Path to the multipage-tiff file-actually works, dont forget to properly save it
    """
    img = Image.open(path)
    images = []
    for i in range(img.n_frames):
        img.seek(i)
        images.append(np.array(img))
    return images

# Loading data

In [13]:
main_path=r"C:\work\Eindhoven_stuff\new_files_from_siebe\OES\Pressure - All 5ns step"
file_path=main_path+"\\"+r"4Abel_N2_66mbar_10kV2021-05-13_13-19-07.tif"

In [22]:
data=read_tif(file_path) #tiff reader if the spectra has multiple frames
data=np.array(data,dtype='float') #change format for massiveOES data compatibility
measured_spectrum=data[4,:,:] # select here which measured frame to process, if you want process all them just put everythin after data loading in for cycle or contact me, i have the scripts for that

In [23]:
plt.imshow(measured_spectrum)

# Reformating to json

In [60]:
"""
In the first step you usually dont know a,b,c,g,l parameters, in that case you wont use create_fit_parameters method.
You take saved json, put it into massiveOES gui, linearize your axis and obtain aprroximate parameters and input them here,
so you dont have to do it again for another similar measurements.
"""

g = 0.15238531491398544 #gauss width
l = 0.00889666100852235 #lorentz width
# y=c*x^2+b*x+a, for linearization of wavelength axis, not necessery to define c,b,a,g or l, it just makes life easier later on
c=385.8911621719568 #start of spectrum
b=0.05394349170652613# #wavelength step
a=1.4426996684765126e-06 #non linear component, quadratic


spectra=OrderedDict() #this is ordered dictionary in which we will put our measured spectra

measured_massive = massiveOES.MeasuredSpectra()   #define massiveOES object


for row in range(measured_spectrum.shape[0]): 
    spectra[row] = {'spectrum':measured_spectrum[row,:]} #create muptiple rows which correspond to spatial dimension of spectrum and input data there



    
measured_massive.spectra = spectra #acces spectra object and input there our measured data


measured_massive.create_fit_parameters(wavelengths=[c, b, a], slitf_gauss=g, slitf_lorentz=l) #put our fitting parameters to massiveOES object
measured_massive.to_json(r'measured_massive_test.json') #save massiveOES object


# Load json file

In [3]:
json_data = massiveOES.MeasuredSpectra.from_json(r'measured_massive_test.json')

In [4]:
specname="100"

In [5]:
data_parameters=json_data.spectra[specname]['params'] #an example of obtaining fitting parameters
# sim_spec=MeasuredSpectra.get_simulated_spec(data,'15') #this is how you get simulated and measured spectra
# meas_spec=MeasuredSpectra.get_measured_spectrum(data,'15') #doesnt work, if the json dont have fitted data!

In [95]:
data_parameters.info

OrderedDict([('species', ['N2CB', 'N2PlusBX'])])

In [104]:
data_parameters.prms

name,value,standard error,relative error,initial value,min,max,vary
wav_start,385.894479,0.00216002,(0.00%),385.8944788834747,-inf,inf,True
wav_step,0.05394349,0.00000000,(0.00%),0.05394349170652613,-inf,inf,False
wav_2nd,1.4427e-06,0.00000000,(0.00%),1.4426996684765126e-06,-inf,inf,False
slitf_gauss,0.15238531,0.00000000,(0.00%),0.15238531491398544,-inf,inf,False
slitf_lorentz,0.00889666,0.00000000,(0.00%),0.00889666100852235,-inf,inf,False
baseline,0.00000000,0.00000000,,0,-inf,inf,False
baseline_slope,0.00000000,0.00000000,,0,-inf,inf,False
N2CB_Trot,410.014352,8.19908490,(2.00%),410.014351925303,300.000000,10000.0000,True
N2CB_Tvib,1814.10403,23.9481710,(1.32%),1814.1040341444634,300.000000,10000.0000,True
N2CB_intensity,0.77289675,0.01430518,(1.85%),0.7728967483067428,0.00000000,inf,True


# Load Databases

In [97]:
"""
Note that if you have loaded json in which you have already added databases for example in gue, they are still there, 
and no further database addition is needed. Put it wont matter if you add the same ones.

Another import point is that if you duplicate your database for example duplicate N2CB.db as N2CB_copy.db and load both of them
it is an easy approach of how to do two temperature fitting which is in a lot of cases better than non-eq fitting.
"""

'\nNote that if you have loaded json in which you have already added databases for example in gue, they are still there, \nand no further database addition is needed. Put it wont matter if you add the same ones.\n\nAnother import point is that if you duplicate your database for example duplicate N2CB.db as N2CB_copy.db and load both of them\nit is an easy approach of how to do two temperature fitting which is in a lot of cases better than non-eq fitting.\n'

In [98]:
#add database of desired species to loaded massiveOES object
First_specie='N2CB.db' #if you want more or less species just just another add_specie function or take one out, also the used_species array has to be adjusted, it is used in nnls fitting
Second_specie='N2PlusBX.db'
N2CB=massiveOES.SpecDB(First_specie)
N2BX=massiveOES.SpecDB(Second_specie)
data_inc_species=[]
used_species=np.array([First_specie[:-3],Second_specie[:-3]])
for specname in json_data.spectra:
    json_data.add_specie(specie=N2CB,specname=specname)
data_inc_species=[]
for specname in json_data.spectra:
    json_data.add_specie(specie=N2BX,specname=specname)
json_data.spectra[specname]['params'].info

c:\work\massiveoes\massiveOES\MeasuredSpectra.py:109: Warning: Specie 'N2CB' not added: Invalid name or already added! See lmfit.Parameters for details.
  warnings.warn(msg,Warning)
c:\work\massiveoes\massiveOES\MeasuredSpectra.py:109: Warning: Specie 'N2PlusBX' not added: Invalid name or already added! See lmfit.Parameters for details.
  warnings.warn(msg,Warning)


OrderedDict([('species', ['N2CB', 'N2PlusBX'])])

# Selection of fitting parameters

In [99]:
#what parameters do you wanna fit my lord?
#set parameters you dont want to fit to FALSE
for specname in json_data.spectra: #specname are the rows of spatial measurement
    
    json_data.spectra['{}'.format(specname)]['params'].prms['wav_start'].vary=True
    json_data.spectra['{}'.format(specname)]['params'].prms['wav_step'].vary=False
    json_data.spectra['{}'.format(specname)]['params'].prms['wav_2nd'].vary=False
    json_data.spectra['{}'.format(specname)]['params'].prms['slitf_gauss'].vary=False
    json_data.spectra['{}'.format(specname)]['params'].prms['slitf_lorentz'].vary=False
    json_data.spectra['{}'.format(specname)]['params'].prms['baseline'].vary=False
    json_data.spectra['{}'.format(specname)]['params'].prms['baseline_slope'].vary=False

In [101]:
#project condition from one line to all other
# it is usufull to speed fitting up, firstly fit one position and extrapolate their parameters to other spatial positions
#sometimes it is better not to transfer fitted parameters from one position to others but do it oteratively
#in that case just take these lines amd input them into fitting cycle (remember to redifine def_v properly)
def_v=100 #position of line from which you want extrapolate fitting parameters
for specname in json_data.spectra:
    
    json_data.spectra['{}'.format(specname)]['params'].prms['wav_start'].value=json_data.spectra['{}'.format(def_v)]['params'].prms['wav_start'].value
    json_data.spectra['{}'.format(specname)]['params'].prms['wav_step'].value=json_data.spectra['{}'.format(def_v)]['params'].prms['wav_step'].value
    json_data.spectra['{}'.format(specname)]['params'].prms['wav_2nd'].value=json_data.spectra['{}'.format(def_v)]['params'].prms['wav_2nd'].value
    json_data.spectra['{}'.format(specname)]['params'].prms['slitf_gauss'].value=json_data.spectra['{}'.format(def_v)]['params'].prms['slitf_gauss'].value
    json_data.spectra['{}'.format(specname)]['params'].prms['slitf_lorentz'].value=json_data.spectra['{}'.format(def_v)]['params'].prms['slitf_lorentz'].value
    json_data.spectra['{}'.format(specname)]['params'].prms['baseline'].value=json_data.spectra['{}'.format(def_v)]['params'].prms['baseline'].value
    json_data.spectra['{}'.format(specname)]['params'].prms['baseline_slope'].value=json_data.spectra['{}'.format(def_v)]['params'].prms['baseline_slope'].value

# Thermal fitting and seperation fo FNS and SPS components

In [103]:
# check how current parameters influence fit
#this is simple thermal fit of one spatial position
fit_check=json_data.fit("100",maxiter=1000)#default maxiter if not defined is 2000
sim_spec_check=massiveOES.MeasuredSpectra.get_simulated_spec(json_data,'100') #this is how you get simulated and measured spectra
meas_spec_check=massiveOES.MeasuredSpectra.get_measured_spectrum(json_data,'100') 
plt.plot(meas_spec_check.x,meas_spec_check.y,".",label="Measurement") #plotting of measurement and simulation, simple .fit function is suficient here
plt.plot(sim_spec_check.x,sim_spec_check.y,"-",label="Simulation")
plt.legend()

********* specname =  100  ************


In [81]:
#seperate fits for two fiting species, here for sps and fns
# if you dont want seperation, just .fit(spcname,maxfv=xxx) is fine, maxfv is number of max number of iterations
set_of_prms_fns_spectra_x={}
set_of_prms_sps_spectra_x={}
set_of_prms_fns_spectra_y={}
set_of_prms_sps_spectra_y={}
prms_array_fns=[]
prms_array_sps=[]
for specname in json_data.spectra:
    if int(specname)<98 or int(specname)>102:
        continue
    fit=json_data.fit(specname,maxiter=1000)
    temp=json_data.spectra['{}'.format(specname)]['params'].prms['N2CB_intensity'].value
    json_data.spectra['{}'.format(specname)]['params'].prms['N2CB_intensity'].value=0
    temp_spec_x=massiveOES.MeasuredSpectra.get_simulated_spec(json_data,specname).x
    temp_spec_y=massiveOES.MeasuredSpectra.get_simulated_spec(json_data,specname).y
    set_of_prms_fns_spectra_x[specname]=temp_spec_x
    set_of_prms_fns_spectra_y[specname]=temp_spec_y
    prms_array_fns.append(json_data.spectra['{}'.format(specname)]['params'])
    json_data.spectra['{}'.format(specname)]['params'].prms['N2CB_intensity'].value=temp

    temp=json_data.spectra['{}'.format(specname)]['params'].prms['N2PlusBX_intensity'].value
    json_data.spectra['{}'.format(specname)]['params'].prms['N2PlusBX_intensity'].value=0
    temp_spec_x=massiveOES.MeasuredSpectra.get_simulated_spec(json_data,specname).x
    temp_spec_y=massiveOES.MeasuredSpectra.get_simulated_spec(json_data,specname).y
    set_of_prms_sps_spectra_x[specname]=temp_spec_x
    set_of_prms_sps_spectra_y[specname]=temp_spec_y
    prms_array_sps.append(json_data.spectra['{}'.format(specname)]['params'])   
    json_data.spectra['{}'.format(specname)]['params'].prms['N2PlusBX_intensity'].value=temp

********* specname =  98  ************


c:\users\kusyl\envs\massive\lib\site-packages\lmfit\minimizer.py:493: RuntimeWarning: ignoring `maxfev` argument to `Minimizer()`. Use `max_nfev` instead.
  warnings.warn(maxeval_warning.format(maxnfev_alias, 'Minimizer'),


********* specname =  99  ************
********* specname =  100  ************
********* specname =  101  ************
********* specname =  102  ************


In [82]:
json_data.spectra['{}'.format(specname)]['params'].prms # here you can see resulting fitting parameters and temperatures
# various info can be also found in print(fit)

name,value,initial value,min,max,vary
wav_start,385.891162,385.8911621719568,-inf,inf,True
wav_step,0.05394349,0.05394349170652613,-inf,inf,False
wav_2nd,1.4427e-06,1.4426996684765126e-06,-inf,inf,False
slitf_gauss,0.15238531,0.15238531491398544,-inf,inf,False
slitf_lorentz,0.00889666,0.00889666100852235,-inf,inf,False
baseline,0.00000000,0,-inf,inf,False
baseline_slope,0.00000000,0,-inf,inf,False
N2CB_Trot,1000.00000,1000.0,300.000000,10000.0000,True
N2CB_Tvib,1000.00000,1000.0,300.000000,10000.0000,True
N2CB_intensity,1.00000000,1,0.00000000,inf,True


In [83]:
sim_spec=massiveOES.MeasuredSpectra.get_simulated_spec(json_data,'100') #this is how you get simulated and measured spectra
meas_spec=massiveOES.MeasuredSpectra.get_measured_spectrum(json_data,'100') 

In [84]:
plt.plot(meas_spec.x,meas_spec.y,"-",label="Measurement") #plotting of measurement and simulation, simple .fit function is suficient here
plt.plot(sim_spec.x,sim_spec.y,".",label="Simulation")
plt.legend()

In [89]:
spec_num="100"

fig, ax = plt.subplots( nrows=1, ncols=1 ) 
ax.plot(meas_spec.x, meas_spec.y,'r.', linewidth=3,label='measurement')
ax.plot(set_of_prms_sps_spectra_x[spec_num], set_of_prms_sps_spectra_y[spec_num], 'b-',label='sps')
ax.plot(set_of_prms_fns_spectra_x[spec_num], set_of_prms_fns_spectra_y[spec_num], 'g-',label='fns')
#ax.plot(my_data_corr.x,(fit_array_neg[iterace]-data_corr.get_measured_spectrum(i).y)-5, color = 'k')
#ax.set_title('Distance from electrode: {0:.4f} mm (pixel:{1})'.format(float(int(i)*imaging_factor),i))
ax.set_xlabel('Wavelength [nm]')
ax.set_ylabel('Relative intensity [a.u.]')
ax.set_ylim(bottom=-12)
plt.legend()
plt.show()

# Non-Thermal fit

In [7]:
"""
Requires same database loading and slection of parameters
"""

'\nRequires same database loading and slection of parameters\n'

In [8]:
json_data.spectra["0"]["params"].info #this is species you are fitting

OrderedDict([('species', ['N2CB', 'N2PlusBX'])])

In [9]:
"""
A dictionary of additional parameters for each fitted specie.
These are:

max_v: *number* the maximal vibrational quantum number allowed

max_J: *number* the maximal rotational quantum number allowed

minlines: *number* the minimal number of lines emitted by transition
           from given upper state in the observed wavelength region.
           This is recommended to be at least 2 to suppress the
           influence of noise.

singlet_like: *bool* if True, the spin-orbit or spin-rotational
              components of the upper states will not be distinguished
                """

'\nA dictionary of additional parameters for each fitted specie.\nThese are:\n\nmax_v: *number* the maximal vibrational quantum number allowed\n\nmax_J: *number* the maximal rotational quantum number allowed\n\nminlines: *number* the minimal number of lines emitted by transition\n           from given upper state in the observed wavelength region.\n           This is recommended to be at least 2 to suppress the\n           influence of noise.\n\nsinglet_like: *bool* if True, the spin-orbit or spin-rotational\n              components of the upper states will not be distinguished\n                '

In [33]:
frame_data={}
frame_spectra={}
for specname in json_data.spectra: #first position takes long, the databases are used to develop matrix in which the data are deposited
    if int(specname)<98 or int(specname)>102:
        continue
    print(specname)
    nnlsfit=json_data.fit_nnls(specname,used_species,maxiter=1,max_J=30,max_v=5) # i think maxiter doesnt work here
    frame_data[specname]= nnlsfit[0]
    frame_spectra[specname]= np.dot(nnlsfit[1], nnlsfit[0].pops)


98
{'maxiter': 1, 'max_J': 30, 'max_v': 5, 'wavelength': 'air'}
{'maxiter': 1, 'max_J': 30, 'max_v': 5, 'wavelength': 'air'}
(301, 493)
dof =  -192
99
{'maxiter': 1, 'max_J': 30, 'max_v': 5, 'wavelength': 'air'}
{'maxiter': 1, 'max_J': 30, 'max_v': 5, 'wavelength': 'air'}
(301, 493)
dof =  -192
100
{'maxiter': 1, 'max_J': 30, 'max_v': 5, 'wavelength': 'air'}
{'maxiter': 1, 'max_J': 30, 'max_v': 5, 'wavelength': 'air'}
(301, 493)
dof =  -192
101
{'maxiter': 1, 'max_J': 30, 'max_v': 5, 'wavelength': 'air'}
{'maxiter': 1, 'max_J': 30, 'max_v': 5, 'wavelength': 'air'}
(301, 493)
dof =  -192
102
{'maxiter': 1, 'max_J': 30, 'max_v': 5, 'wavelength': 'air'}
{'maxiter': 1, 'max_J': 30, 'max_v': 5, 'wavelength': 'air'}
(301, 493)
dof =  -192


In [37]:
meas_spec=massiveOES.MeasuredSpectra.get_measured_spectrum(json_data,'100') 

In [43]:
specname="100"
plt.plot(meas_spec.x,meas_spec.y,"-",label="Measurement") #plotting of measurement and simulation, simple .fit function is suficient here
plt.plot(meas_spec.x,frame_spectra[specname],".",label="Simulation")
plt.legend()

In [51]:
frame_data[specname]

,id,E_J,J,component,E_v,v,numlines,pops,errors,specie,degeneracy
0,57,0.000012,19.0,3,0.000456,0,1,0.000000,196.130966,N2CB,39.0
1,59,0.000012,20.0,2,0.000456,0,1,0.000000,1920.461032,N2CB,41.0
2,60,0.000014,20.0,3,0.000456,0,1,0.000000,4703.602078,N2CB,41.0
3,62,0.000013,21.0,2,0.000456,0,1,0.000000,2343.931613,N2CB,43.0
4,63,0.000015,21.0,3,0.000456,0,1,0.105854,5224.728839,N2CB,43.0
...,...,...,...,...,...,...,...,...,...,...,...
488,578,0.000025,27.5,2,0.000410,0,3,0.000000,247.381214,N2PlusBX,56.0
489,579,0.000027,28.5,1,0.000410,0,3,0.000000,147.657492,N2PlusBX,58.0
490,580,0.000027,28.5,2,0.000410,0,3,0.000000,504.455390,N2PlusBX,58.0
491,581,0.000029,29.5,1,0.000410,0,3,0.000000,15.896841,N2PlusBX,60.0


# Population distribution

In [47]:
specname="100"

In [48]:
probed_frame=frame_data[specname]

In [52]:
probed_frame['E_v'] *= 100 * h * c / e
probed_frame['E_J'] *= 100 * h * c / e

probed_frame['degeneracy']=2*probed_frame['J']+1 #creates new column named degeneracy

In [54]:
vib_len=np.unique(probed_frame['v'])# how many vib states are considered in fit
print(vib_len)

[0 1 2 3 4]


In [57]:
"""
Creates x number fo plots where x represents number of fitted vibrational states defined by vib_len.
In plots are distributions of populations against energies.
"""
vib_len=np.unique(probed_frame['v'])
bands=np.unique(probed_frame['specie'])
for vib in vib_len:
    temp_vib_num=probed_frame.loc[probed_frame['v'] == vib]
    for band in bands:
        if band == 'N2CB':
            band_num=temp_vib_num.loc[probed_frame['specie'] == band]
            band_col='b'
            plt.figure(vib)
            plt.title('Vib band {}'.format(vib))
            plt.plot(band_num['E_J']+band_num['E_v'],band_num.pops/(band_num.degeneracy),'{}.'.format(band_col))
            plt.yscale('log')
            plt.show()
        if band == 'N2PlusBX':
            band_num=temp_vib_num.loc[probed_frame['specie'] == band]
            band_col='r'
            plt.figure(vib)
            plt.title('Vib band {}'.format(vib))
            plt.plot(band_num['E_J']+band_num['E_v'],band_num.pops/(band_num.degeneracy),'{}.'.format(band_col))
            plt.yscale('log')
            plt.show()

# Few notes